# Sequence Comparison of DNA using `BioPython`
Comparison of `Covid19`, `MERS`, `SARS` and `EBOLA`

#### Analysis Techniques:
* Compare their DNA sequence and Protein (Amino Acid) sequence
* GC Content
* Freq of Each Amino Acids
* Find similarity between them
  * Alignment
  * hamming distance
* 3D structure of each

| DNA Sequence     | Datasource                                                    |
|:-----------------|:--------------------------------------------------------------|
| Latest Sequence  | https://www.ncbi.nlm.nih.gov/genbank/sars-cov-2-seqs/         |
| Wuhan-Hu-1       | https://www.ncbi.nlm.nih.gov/nuccore/MN908947.3?report=fasta  |
| Covid19          | https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2?report=fasta |
| SARS             | https://www.ncbi.nlm.nih.gov/nuccore/NC_004718.3?report=fasta |
| MERS             | https://www.ncbi.nlm.nih.gov/nuccore/NC_019843.3?report=fasta |
| EBOLA            | https://www.ncbi.nlm.nih.gov/nuccore/NC_002549.1?report=fasta |


In [2]:
from Bio.Seq import Seq

# Create our sequence 
seq1 = Seq('ACTCGA')
seq2 = Seq('AC')


#### GC Contents In DNA  

* `GC-content` (or guanine-cytosine content) is the **percentage of nitrogenous bases** in a DNA or RNA molecule that are either guanine (`G`) or cytosine (`C`)

#### Usefulness  

* In polymerase chain reaction (PCR) experiments, the GC-content of short oligonucleotides known as primers is often used to predict their **annealing temperature** to the template DNA.
    * A `high` GC-content level indicates a relatively higher melting temperature.
    * DNA with `low` GC-content is less stable than DNA with high GC-content  

> Question: which sequence is more stable when heat is applied?

In [46]:
from Bio.SeqUtils import GC
# Check GC (guanine-cytosine) percentage in sequence

print(f"{GC(seq1) = }% \t({seq1})")
print(f"{GC(seq2) = }% \t({seq2})")

GC(seq1) = 50.0% 	(ACTCGA)
GC(seq2) = 50.0% 	(AC)


### Sequence Alignments
* `Global alignment` finds the best concordance/agreement between all characters in two sequences
* `Local Alignment` finds just the subsequences that align the best

In [103]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

print('seq1 =', seq1, '\nseq2 =', seq2, '\n\n')

# Global alignment
alignments = pairwise2.align.globalxx(seq1, seq2)

print(f'Alignments found: {len(alignments)}')
print(*alignments)

seq1 = ACTCGA 
seq2 = AC 

Alignments found: 2
('ACTCGA', 'A--C--', 2.0, 0, 6)
('ACTCGA', 'AC----', 2.0, 0, 6)


In [104]:
# Print nicely
print(format_alignment(*alignments[0]))

ACTCGA
|  |  
A--C--
  Score=2


In [105]:
# 2nd alignment
print(format_alignment(*alignments[1]))

ACTCGA
||    
AC----
  Score=2


In [106]:
# To see all possible alignments
for a in alignments:
    print(format_alignment(*a), '\n')

ACTCGA
|  |  
A--C--
  Score=2
 
ACTCGA
||    
AC----
  Score=2
 


In [107]:
# Get the number of possible sequence alignments
alignment_score = pairwise2.align.globalxx(seq1,seq2,one_alignment_only=True,score_only=True)
alignment_score

2.0

#### Sequence Similarity
* Fraction of nucleotides that is the same/ total number of nucleotides * 100%

In [108]:
alignment_score/len(seq1)*100

33.33333333333333

### Hamming Distance: `How Many Subsitutions are Required to Match Two Sequences?`

* Hamming distance between two strings of equal length is the number of positions at which the corresponding symbols are different.
* In other words, it measures the minimum number of substitutions required to change one string into the other, or the minimum number of errors that could have transformed one string into the other
* It is used for error detection or error correction
* It is used to quantify the similarity of DNA sequences  

#### Edit Distance
   * Is a way of quantifying how dissimilar two strings (e.g., words) are to one another by counting the minimum number of operations required to transform one string into the other (e.g. Levenshtein distance)

In [109]:
def hamming_distance(lhs, rhs):
    return len([(x,y) for x,y in zip(lhs,rhs) if x != y])

In [133]:
hamming_distance('TT', 'ACCTA')

2

In [183]:
def hammer_time(s1, s2, verbose=True):
    """Take two nucleotide sequences s1 and s2, and display 
       the possible alignments and hamming distance.
    """
    if verbose:
        print('s1 =', s1, '\ns2 =', s2, '\n\n')
        print('Hamming Distance:', hamming_distance(s1, s2), '\n(min substitutions for sequences to match)')
        print('\nAlignment Options:\n\n')
    alignments = pairwise2.align.globalxx(s1, s2)
    for a in alignments:
        print(format_alignment(*a), '\n')
    

s1 = 'ACTCGAA'
s2 = 'ACGA'
hammer_time(s1, s2)

s1 = ACTCGAA 
s2 = ACGA 

Hamming Distance: 2 
(min substitutions for sequences to match)

Alignment Options:

ACTCGAA
|  || |
A--CG-A
  Score=4
 
ACTCGAA
||  | |
AC--G-A
  Score=4
 
ACTCGAA
|  ||| 
A--CGA-
  Score=4
 
ACTCGAA
||  || 
AC--GA-
  Score=4
 


### Dot Plot
* A dot plot is a graphical method that allows the **comparison of two biological sequences** and identify regions of **close similarity** between them.
* Simplest explanation: put a dot wherever sequences are identical

#### Usefulness
Dot plots can also be used to visually inspect sequences for
    - Direct or inverted repeats
    - Regions with low sequence complexity
    - Similar regions
    - Repeated sequences
    - Sequence rearrangements
    - RNA structures
    - Gene order

Acknowledgement: https://stackoverflow.com/questions/40822400/how-to-create-a-dotplot-of-two-dna-sequence-in-python

In [177]:
def delta(x,y):
    return 0 if x == y else 1

def M(seq1,seq2,i,j,k):
    return sum(delta(x,y) for x,y in zip(seq1[i:i+k],seq2[j:j+k]))

def makeMatrix(seq1,seq2,k):
    n = len(seq1)
    m = len(seq2)
    return [[M(seq1,seq2,i,j,k) for j in range(m-k+1)] for i in range(n-k+1)]

def plotMatrix(M,t, seq1, seq2, nonblank = chr(0x25A0), blank = ' '):
    print(' |' + seq2)
    print('-'*(2 + len(seq2)))
    for label,row in zip(seq1,M):
        line = ''.join(nonblank if s < t else blank for s in row)
        print(label + '|' + line)

def dotplot(seq1,seq2,k = 1,t = 1):
    M = makeMatrix(seq1,seq2,k)
    plotMatrix(M, t, seq1,seq2) #experiment with character choice

In [178]:
# The dot plot: put a dot where the two sequences are identical

s1 = 'ACTCGA'
s2 = 'AC'

dotplot(s1, s2)

 |AC
----
A|■ 
C| ■
T|  
C| ■
G|  
A|■ 


In [199]:
#  Identical proteins will show a diagonal line.
s1 = 'ACCTAG'
s2 = 'ACCTAG'

dotplot(s1, s2)

print('\n\n')
hammer_time(s1, s2, verbose=False)

 |ACCTAG
--------
A|■   ■ 
C| ■■   
C| ■■   
T|   ■  
A|■   ■ 
G|     ■


ACCTAG
||||||
ACCTAG
  Score=6
 
